# Cuaderno 19: Problema del conjunto estable de peso máximo

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados: 
* un grafo no dirigido $G=(V,E)$; y,
* un vector $w \in \ZZ^{V}$ de pesos asociados a los nodos de $G$.

Un *conjunto estable* en G es un conjunto de nodos $S \subset V$ mutuamente no adyacentes entre sí, es decir, con la propiedad de que ninguna arista del grafo tiene ambos extremos en $S$. El problema del conjunto estable de peso máximo consiste en encontrar un conjunto estable en G con el mayor peso posible, donde el peso de un conjunto $S$ se define como la suma de los pesos de los nodos contenidos en el mismo. 

Utilizando variables binarias $x_i$ para la selección de nodos, el problema del conjunto estable de peso máximo puede formularse como el siguiente programa lineal entero:

\begin{align*}
\max &\sum_{i \in V} w_{i} x_{i}\\ 
& \mbox{s.r.}\\
& x_i + x_j \leq 1 , \quad \forall ij \in E,\\
& x_{i} \in \{0, 1\}, \quad \forall i \in V.
\end{align*}

La función objetivo mide el peso del conjunto de nodos seleccionado.

La primera familia de restricciones establece que ningún par de nodos conectados con una arista pueden ser seleccionados a la vez.

Vamos a implementar este modelo usando la interfaz Python de Gurobi.



Definimos primero los datos. Al tratarse de un grafo no dirigido, orientamos arbitrariamente cada arista de $E$ (por ejemplo, poniendo primero el nodo con el menor índice.

In [ ]:
from gurobipy import *
# from random import randint, random

# Nodos del grafo y sus pesos
V, w = multidict(tupledict({
     1 : 5, 2 : 6, 3 : 6,
     4 : 5, 5 : 5, 6 : 4,
     7 : 6, 8 : 5, 9 : 4, 10 : 6}))

# Aristas del grafo
E = tuplelist([(1, 2), (1, 3),
               (2, 4), (2, 5),
               (3, 4), (3, 6),
               (5, 6), (1, 7),
               (3, 7), (5, 7),
               (2, 8), (4, 8),
               (5, 8), (3, 9),
               (6, 9), (7, 9),
               (3, 10), (4, 10),
               (8, 10)]) 


# El siguiente código genera un grafo aleatorio con n nodos
# n = 1000
# V = range(1, n+1)
# w = {i : randint(6, 10) for i in V} 
# E = [(i,j) for i in V for j in V if i <j and random()<0.5]
print(list(V))
print(w)
print(E)


Podemos dibujar esta instancia del problema empleando los módulos `ipycytoscape` y `networkx`:

In [ ]:
import networkx as nx
import ipycytoscape
D = nx.Graph()
D.add_nodes_from(V)
for i in V:
    D.nodes[i]['etiq']= '{}\n{}'.format(i, w[i])
    D.nodes[i]['color'] =  '#9dbaea' #if vx[i,j]<=0.1 else '#ff007f'
D.add_edges_from(E)
grafo = ipycytoscape.CytoscapeWidget()
grafo.graph.add_graph_from_networkx(D, directed=False)
grafo.set_style([{'selector': 'node', 'style' : {'background-color': 'data(color)', 'font-family': 'helvetica', 'font-size': '10px', 'color':'white', 'label': 'data(etiq)', 'text-wrap' : 'wrap', 'text-valign' : 'center'}}, 
                    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}, 'style' : {'font-family': 'helvetica', 'font-size': '10px', 'label': 'data(etiq)'}}, 
                    {'selector': 'edge', 'style': {'width': 4, 'line-color': '#9dbaea', 'font-size': '10px', 'label': 'data(etiq)', 'text-valign' : 'top', 'text-margin-y' : '-10px'}}, 
                    {'selector': 'edge.directed', 'style': {'curve-style': 'bezier', 'target-arrow-shape': 'triangle', 'target-arrow-color': 'data(color)'}}])
grafo

Definimos ahora el objeto modelo, las variables y la función objetivo.

In [ ]:
# Crear el objeto modelo
m = Model('maxstab')

# Crear las variables de seleccion de nodos
x = m.addVars(V, name="x", vtype=GRB.BINARY)

# Crear la funcion objetivo
m.setObjective(x.prod(w,'*'), GRB.MAXIMIZE)


Añadimos las restricciones de conflictos de aristas:

In [ ]:
# Restricciones de aristas
m.addConstrs((x[i] + x[j]  <= 1 for i,j in E), 
                 "arista")


Escribimos el modelo a un archivo de texto:

In [ ]:
# Escribir el modelo a un archivo
m.write('maxstab.lp')


Finalmente, resolvemos el modelo y mostramos la solución:

In [ ]:
# Calcular la solucion optima
m.optimize()

# Escribir la solucion
if m.status == GRB.Status.OPTIMAL:
    # Recuperar los valores de las variables
    vx = m.getAttr('x', x)
    print('\nConjunto estable de peso maximo:')
    print([i for i in V if vx[i]>=0.9])


Grafiquemos ahora esta solución empleando los módulos `networkx` y `ipycytoscape`:

In [ ]:
import networkx as nx
import ipycytoscape
D = nx.Graph()
D.add_nodes_from(V)
for i in V:
    D.nodes[i]['etiq']= '{}\n{}'.format(i, w[i])
    D.nodes[i]['color'] =  '#9dbaea' if vx[i]<=0.1 else '#ff007f'
D.add_edges_from(E)
grafo = ipycytoscape.CytoscapeWidget()
grafo.graph.add_graph_from_networkx(D, directed=False)
grafo.set_style([{'selector': 'node', 'style' : {'background-color': 'data(color)', 'font-family': 'helvetica', 'font-size': '10px', 'color':'white', 'label': 'data(etiq)', 'text-wrap' : 'wrap', 'text-valign' : 'center'}}, 
                    {'selector': 'node:parent', 'css': {'background-opacity': 0.333}, 'style' : {'font-family': 'helvetica', 'font-size': '10px', 'label': 'data(etiq)'}}, 
                    {'selector': 'edge', 'style': {'width': 4, 'line-color': '#9dbaea', 'font-size': '10px', 'label': 'data(etiq)', 'text-valign' : 'top', 'text-margin-y' : '-10px'}}, 
                    {'selector': 'edge.directed', 'style': {'curve-style': 'bezier', 'target-arrow-shape': 'triangle', 'target-arrow-color': 'data(color)'}}])
grafo

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Implementación de modelos lineales enteros
# Problema del conjunto estable de peso maximo

# Luis M. Torres (EPN 2021)

from gurobipy import *
import networkx as nx
import ipycytoscape
from random import randint,random

# El siguiente código genera un grafo aleatorio con n nodos
n = 400
V = range(1, n+1)
w = {i : randint(6, 10) for i in V} 
E = [(i,j) for i in V for j in V if i <j and random()<0.5]


# Nodos del grafo y sus pesos
# V, w = multidict(tupledict({
#     1 : 5, 2 : 6, 3 : 6,
#     4 : 5, 5 : 5, 6 : 4,
#     7 : 6, 8 : 5, 9 : 4, 10 : 6}))

# Aristas del grafo
# E = tuplelist([(1, 2), (1, 3),
#                (2, 4), (2, 5),
#                (3, 4), (3, 6),
#                (5, 6), (1, 7),
#                (3, 7), (5, 7),
#                (2, 8), (4, 8),
#                (5, 8), (3, 9),
#                (6, 9), (7, 9),
#                (3, 10), (4, 10),
#                (8, 10)]) 

try:
    # Crear el objeto modelo
    m = Model('maxstab')

    # Crear las variables de seleccion de nodos
    x = m.addVars(V, name="x", vtype=GRB.BINARY)

    # Crear la funcion objetivo
    m.setObjective(x.prod(w,'*'), GRB.MAXIMIZE)

    # Restricciones de aristas
    m.addConstrs((x[i] + x[j]  <= 1 for i,j in E), 
                 "arista")

    # Escribir el modelo a un archivo
    m.write('maxstab.lp')

    # Calcular la solucion optima
    m.optimize()

    # Escribir la solucion
    if m.status == GRB.Status.OPTIMAL:
        # Recuperar los valores de las variables
        vx = m.getAttr('x', x)
        print('\nConjunto estable de peso maximo:')
        print ([i for i in V if vx[i]>=0.9])
        
    # Graficar la solución
    # D = nx.Graph()
    # D.add_nodes_from(V)
    # for i in V:
    #     D.nodes[i]['etiq']= '{}\n{}'.format(i, w[i])
    #     D.nodes[i]['color'] =  '#9dbaea' if vx[i]<=0.1 else '#ff007f'
    # D.add_edges_from(E)
    # grafo = ipycytoscape.CytoscapeWidget()
    # grafo.graph.add_graph_from_networkx(D, directed=False)
    # grafo.set_style([{'selector': 'node', 'style' : {'background-color': 'data(color)', 'font-family': 'helvetica', 'font-size': '10px', 'color':'white', 'label': 'data(etiq)', 'text-wrap' : 'wrap', 'text-valign' : 'center'}}, 
    #                 {'selector': 'node:parent', 'css': {'background-opacity': 0.333}, 'style' : {'font-family': 'helvetica', 'font-size': '10px', 'label': 'data(etiq)'}}, 
    #                 {'selector': 'edge', 'style': {'width': 4, 'line-color': '#9dbaea', 'font-size': '10px', 'label': 'data(etiq)', 'text-valign' : 'top', 'text-margin-y' : '-10px'}}, 
    #                 {'selector': 'edge.directed', 'style': {'curve-style': 'bezier', 'target-arrow-shape': 'triangle', 'target-arrow-color': 'data(color)'}}])
    
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')

In [ ]:
grafo